In [5]:
import re

file = open('input1.txt', 'r')
regex = '(1|2|3|4|5|6|7|8|9|0|one|two|three|four|five|six|seven|eight|nine)'
translation = {'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}

sum_digits = 0
sum_digits_and_text = 0

for line in file.readlines():
    matches = []
    for i in range(len(line)):
        match = re.search(regex, line[i:])
        if match is not None:
            matches.append(match.group(0))

    first = -1
    first_digit = -1

    last = -1
    last_digit = -1

    for match in matches:
        digit = 0
        try:
            digit = int(match)
            if first_digit == -1:
                first_digit = digit
            last_digit = digit
        except ValueError:
            digit = translation[match]

        if first == -1:
            first = digit
        last = digit

    if first == -1 or last == -1:
        print(matches)
        print(first)
        print(last)
        
    sum_digits += 10*first_digit + last_digit
    sum_digits_and_text += 10*first + last

print(f'The sum of the first and last digit on each line is {sum_digits}')
print(f'The sum of the first and last digit on each line (including text) is {sum_digits_and_text}')
        

The sum of the first and last digit on each line is 53386
The sum of the first and last digit on each line (including text) is 53312


In [6]:
import re
file = open('input2.txt')

max_red = 12
max_green = 13
max_blue = 14

impossible = []
possible = []
powers = []

for line in file.readlines():
    game_match = re.match('Game ([0-9]+): (.+)$', line)
    id = int(game_match.group(1))
    draws = game_match.group(2).split(';')

    min_red = 0
    min_green = 0
    min_blue = 0
    
    for draw in draws:
        red_match = re.search('([0-9]+) red', draw)
        green_match = re.search('([0-9]+) green', draw)
        blue_match = re.search('([0-9]+) blue', draw)

        red = 0 if red_match is None else int(red_match.group(1))
        green = 0 if green_match is None else int(green_match.group(1))
        blue = 0 if blue_match is None else int(blue_match.group(1))

        min_red = red if red > min_red else min_red
        min_green = green if green > min_green else min_green
        min_blue = blue if blue > min_blue else min_blue

        if id not in impossible and (red > max_red or green > max_green or blue > max_blue):
            impossible.append(id)

    if id not in impossible:
        possible.append(id)

    powers.append(min_red*min_green*min_blue)

print(f'The sum of the IDs of the games that were possible is {sum(possible)}')
print(f'The sum of the powers of the minimum sets of cubes is {sum(powers)}')
        

The sum of the IDs of the games that were possible is 2406
The sum of the powers of the minimum sets of cubes is 78375


In [7]:
import re
import numpy

file = open('input3.txt')
schematic = []
part_numbers = []

gears = {}

for line in file.readlines():
    schematic.append(line.strip())

running_number = ''
adjacent_gears = set()
adjacent = False

for i in range(len(schematic)):
    for j in range(len(schematic[i])):
        c = schematic[i][j]
        if c.isdigit():
            running_number += c

            # Check nearby cells for symbols
            for k in range(-1, 2):
                for l in range(-1, 2):
                    try:
                        other_c = schematic[i+k][j+l]
                        if re.search('[^\.0-9]', other_c) is not None:
                            adjacent = True
                        if other_c == '*':
                            adjacent_gears.add((i+k, j+l))
                    except IndexError:
                        pass

        else:
            if adjacent:
                part_numbers.append(int(running_number))
                for gear in adjacent_gears:
                    if gear not in gears:
                        gears[gear] = []
                    gears[gear].append(int(running_number))

            running_number = ''
            adjacent_gears = set()
            adjacent = False

print(f'The sum of all the part numbers is {sum(part_numbers)}')
print(f'The sum of all gear ratios is {sum(numpy.prod(gears[gear]) for gear in gears if len(gears[gear]) == 2)}')

The sum of all the part numbers is 539637
The sum of all gear ratios is 82818007


In [8]:
import re

file = open('input4.txt', 'r')
cards = []
for line in file.readlines():
    match = re.search('Card +([0-9]+): +(.*)', line)
    winning_numbers, card_numbers = match.group(2).split('|')

    winning_numbers = winning_numbers.split(' ')
    winning_numbers = [int(number) for number in winning_numbers if number.isdigit()]

    card_numbers = card_numbers.split(' ')
    card_numbers = [int(number) for number in card_numbers if number.isdigit()]

    cards.append({
        'winning_numbers': winning_numbers,
        'card_numbers': card_numbers
    })

points = 0
for card in cards:
    count = 0
    for number in card['card_numbers']:
        if number in card['winning_numbers']:
            count += 1
    if count > 0:
        points += 2**(count-1)

    card['count'] = count

print(f'The sum of scratch card points is {points}')

number_of_cards = [1 for card in cards]
for i in range(len(cards)):
    count = cards[i]['count']
    for j in range(count):
        number_of_cards[i+j+1] += number_of_cards[i]

print(f'The total number of scratch cards at the end is {sum(number_of_cards)}')

The sum of scratch card points is 19135
The total number of scratch cards at the end is 5704953


In [29]:
inputs = open('input5.txt', 'r').read().split('\n\n')

# Start with a map from the seed ranges to itself
seeds = inputs[0].split()[1:]
maps = []
for i in range(0, len(seeds), 2):
    start_range = int(seeds[i])
    length = int(seeds[i+1])
    end_range = start_range + length - 1
    maps.append({'source_range': (start_range, end_range), 'transformation': 0})
    # Add single number ranges to solve part A
    maps.append({'source_range': (length, length), 'transformation': 0})

# Go through the mappings one at a time, and append them to the total map
for mapping in inputs[1:]:
    new_maps = []
    for single_map in mapping.strip().split('\n')[1:]:
        target, source, length = [int(x) for x in single_map.split()]
        source_range = (source, source + length - 1)
        transformation = target - source
        new_maps.append({'source_range': source_range, 'transformation': transformation})

    maps = append_maps(maps, new_maps)

def apply_map(start_value, maps):
    for map in maps:
        if map['source_range'][0] <= start_value <= map['source_range'][1]:
            return start_value + map['transformation']
    return start_value

print(f'The lowest reachable location taking the seeds as discrete locations if {min(apply_map(int(seed), maps) for seed in seeds)}')
print(f'The lowest reachable location is {min(map["source_range"][0] + map["transformation"] for map in maps)}')

# Create a set of maps from seed to location, by appending each intermediate map to the running total of maps
def append_maps(original_maps, appended_maps):
    result_maps = []
    has_intersections = True

    while has_intersections:
        has_intersections = False
        for i in range(len(original_maps)):
            original_map = original_maps[i]
            transformation = original_map['transformation']
            target_range = tuple(x + transformation for x in original_map['source_range'])

            for appended_map in appended_maps:
                intersection, non_intersected_sections = find_intersections(target_range, appended_map['source_range'])
                if intersection is not None:
                    has_intersections = True
                    del original_maps[i]

                    appended_transformation = appended_map['transformation']
                    new_source_range = tuple(x - transformation for x in intersection)
                    result_maps.append({'source_range': new_source_range, 'transformation': transformation + appended_transformation})
                    for section in non_intersected_sections:
                        section_range = tuple(x - transformation for x in section)
                        original_maps.append({'source_range': section_range, 'transformation': transformation})
                    break

            if has_intersections:
                break

    # If anything remains in the original maps list at this point, it had no overlaps with the appended_maps
    for original_map in original_maps:
        result_maps.append(original_map)
    return result_maps

def find_intersections(range1, range2):
    intersection = (max(range1[0], range2[0]), min(range1[1], range2[1]))
    if intersection[1] < intersection[0]:
        intersection = None
    non_intersecting = set()
    if range1[0] < range2[0]:
        non_intersecting.add((range1[0], min(range1[1], range2[0]-1)))
    if range2[1] < range1[1]:
        non_intersecting.add((max(range1[0], range2[1]+1), range1[1]))

    return intersection, non_intersecting

The lowest reachable location taking the seeds as discrete locations if 662197086
The lowest reachable location is 52510809


In [5]:
import numpy
import math

def find_wins(time, record):
    return time - 2 * (math.floor((time - math.sqrt(time**2 - 4 * record)) / 2 + 1)) + 1

file = open('input6.txt', 'r')
times_string, distances_string = file.readlines()
times = [int(x) for x in times_string.split() if x.isdigit()]
distances = [int(x) for x in distances_string.split() if x.isdigit()]

wins = [find_wins(time, record) for time, record in zip(times, distances)]
print(f'The product of the numbers of ways to win the short races is {numpy.prod(wins)}')

# One long race
time = int(''.join(x for x in times_string if x.isdigit()))
record = int(''.join(x for x in distances_string if x.isdigit()))
print(f'The number of ways to win the really long race is {find_wins(time, record)}')

The product of the numbers of ways to win the short races is 1624896
The number of ways to win the really long race is 32583852


In [32]:
import functools

hands = [line.split(' ') for line in open('input7.txt', 'r').readlines()]
hands = [(x, int(y)) for x,y in hands]

card_values = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
with_joker_card_values = ['A', 'K', 'Q', 'T', '9', '8', '7', '6', '5', '4', '3', '2', 'J']

def get_hand_score(hand):
    cards = {}
    for card in hand[0]:
        if card not in cards:
            cards[card] = 0
        cards[card] += 1

    # Check for five of a kind
    if 5 in cards.values():
        return 19
    if 4 in cards.values():
        return 18
    if 3 in cards.values() and 2 in cards.values():
        return 17
    if 3 in cards.values():
        return 16

    pairs = sum(1 if value == 2 else 0 for value in cards.values())
    if pairs == 2:
        return 15
    if pairs == 1:
        return 14

    return 13

def compare_hands(hand1, hand2):
    comparison = get_hand_score(hand1) - get_hand_score(hand2)

    card_index = 0
    while comparison == 0 and card_index < 5:
        comparison = card_values.index(hand2[0][card_index]) - card_values.index(hand1[0][card_index])
        card_index += 1
    return comparison

def get_joker_hand_score(hand):
    cards = {}
    jokers = 0
    for card in hand[0]:
        if card == 'J':
            jokers += 1
            continue
            
        if card not in cards:
            cards[card] = 0
        cards[card] += 1

    if max(cards.values(), default=0) + jokers == 5:
        return 19
    if max(cards.values(), default=0) + jokers == 4:
        return 18

    pairs = sum(1 if value == 2 else 0 for value in cards.values())
    if (3 in cards.values() and 2 in cards.values()) or (pairs == 2 and jokers == 1):
        return 17

    if max(cards.values(), default=0) + jokers == 3:
        return 16

    if pairs == 2:
        return 15

    if pairs == 1 or jokers == 1:
        return 14

    return 13

def compare_joker_hands(hand1, hand2):
    comparison = get_joker_hand_score(hand1) - get_joker_hand_score(hand2)

    card_index = 0
    while comparison == 0 and card_index < 5:
        comparison = with_joker_card_values.index(hand2[0][card_index]) - with_joker_card_values.index(hand1[0][card_index])
        card_index += 1
    return comparison

hands.sort(key=functools.cmp_to_key(compare_hands))

winnings = 0
for index, hand in enumerate(hands):
    winnings += (index+1) * hand[1]
print(f'The total winnings for the hands is {winnings}')

hands.sort(key=functools.cmp_to_key(compare_joker_hands))

winnings = 0
for index, hand in enumerate(hands):
    winnings += (index+1) * hand[1]
print(f'The total winnings for the hands, including jokers, is {winnings}')

The total winnings for the hands is 249638405
The total winnings for the hands, including jokers, is 249776650


In [1]:
import re, math
instruction_search = '(\w+) = \((\w+), (\w+)\)'
instructions = open('input8.txt', 'r').read().split('\n')

directions = [0 if c == 'L' else 1 for c in instructions[0]]
nodes = {match.group(1): [match.group(2), match.group(3)] for instruction in instructions if (match := re.search(instruction_search, instruction))}

current_node = 'AAA'
steps = 0

while current_node != 'ZZZ':
    current_node = nodes[current_node][directions[steps % len(directions)]]
    steps += 1
print(f'It took {steps} steps to get to ZZZ')

ghost_nodes = [node for node in nodes if re.match('..A', node)]
steps = 0

# Find cycles
def find_cycle(node):
    current_node = node
    history = []
    current_step = 0

    def find_cycle():
        cycles = 1
        while cycles*len(directions) < len(history):
            if history[-1-cycles*len(directions)] == history[-1]:
                return cycles*len(directions)
            cycles += 1
        return -1

    while (cycle_length := find_cycle()) == -1:
        history.append(current_node)
        current_node = nodes[current_node][directions[current_step % len(directions)]]
        current_step += 1

    # Find the Zs witin the cycle
    targets_in_cycle = []
    for i in range(cycle_length):
        if re.search('..Z', history[-1-cycle_length+i]):
            targets_in_cycle.append(i)

    return cycle_length, current_step - cycle_length - 1, targets_in_cycle

cycles = [find_cycle(ghost_node) for ghost_node in ghost_nodes]

# This task has been made in order to make this easier
lcm = 1
for cycle in cycles:
    lcm = math.lcm(lcm, cycle[0])
print(f'The number of ghost steps to take is {lcm}')

714
It took 22411 steps to get to ZZZ
The number of ghost steps to take is 11188774513823


In [17]:
sequences = [[int(x) for x in line.split()] for line in open('input9.txt', 'r').readlines()]

def reduce_sequence(sequence):
    return [sequence[i] - sequence[i-1] for i in range(1,len(sequence))]

def expand_sequence_backwards(expanding_sequence, lower_sequence):
    if len(lower_sequence) != len(expanding_sequence):
        raise Exception('ME NO WORKY')

    expanded_sequence = [x for x in expanding_sequence]
    expanded_sequence.append(expanding_sequence[-1] + lower_sequence[-1])
    return expanded_sequence

def expand_sequence_forwards(expanding_sequence, lower_sequence):
    if len(lower_sequence) != len(expanding_sequence):
        raise Exception('NO WORKY WORSE')

    expanded_sequence = [x for x in expanding_sequence]
    expanded_sequence.insert(0, expanding_sequence[0] - lower_sequence[0])
    return expanded_sequence

def is_all_zero(sequence):
    for x in sequence:
        if x != 0:
            return False
    return True

expanded_sequences = []
for sequence in sequences:
    altered_sequences = [[x for x in sequence]]
    depth = 0
    while not is_all_zero(altered_sequences[depth]):
        altered_sequences.append(reduce_sequence(altered_sequences[depth]))
        depth += 1

    altered_sequences[depth].append(0)

    for i in range(depth, 0, -1):
        altered_sequences[i - 1] = expand_sequence_backwards(altered_sequences[i - 1], altered_sequences[i])

    altered_sequences[depth].append(0)
    for i in range(depth, 0, -1):
        altered_sequences[i - 1] = expand_sequence_forwards(altered_sequences[i - 1], altered_sequences[i])
    
    expanded_sequences.append(altered_sequences[0])

print(f'The sum of all extrapolated values is {sum(expanded_sequence[-1] for expanded_sequence in expanded_sequences)}')
print(f'The sum of all forward extrapolated values is {sum(expanded_sequence[0] for expanded_sequence in expanded_sequences)}')

The sum of all extrapolated values is 1789635132
The sum of all forward extrapolated values is 913


In [1]:
lines = open('input10.txt', 'r').readlines()
connections = {}
start = None

size = (len(lines), len(lines[0]))

for row, line in enumerate(lines):
    for column, character in enumerate(line):
        if character == 'S':
            start = (row, column)
        elif character == '.':
            continue
        elif character == 'F':
            connections[(row, column)] = [(row+1, column), (row, column+1)]
        elif character == '7':
            connections[(row, column)] = [(row+1, column), (row, column-1)]
        elif character == 'J':
            connections[(row, column)] = [(row, column-1), (row-1, column)]
        elif character == 'L':
            connections[(row, column)] = [(row-1, column), (row, column+1)]
        elif character == '-':
            connections[(row, column)] = [(row, column-1), (row, column+1)]
        elif character == '|':
            connections[(row, column)] = [(row-1, column), (row+1, column)]

start_connections = []
for i in range(-1, 2, 1):
    for j in range(-1, 2, 1):
        test_coordinate = (start[0] + i, start[1] + j)
        if test_coordinate in connections and start in connections[test_coordinate]:
            start_connections.append(test_coordinate)

# Replace the start icon
above = (start[0]-1, start[1])
right = (start[0], start[1]+1)
below = (start[0]+1, start[1])
left = (start[0], start[1]-1)

if above in start_connections and right in start_connections:
    start_sign = 'L'
elif above in start_connections and below in start_connections:
    start_sign = '|'
elif above in start_connections and left in start_connections:
    start_sign = 'J'
elif right in start_connections and below in start_connections:
    start_sign = 'F'
elif right in start_connections and left in start_connections:
    start_sign = '-'
elif below in start_connections and left in start_connections:
    start_sign = '7'

loop = set()
current = start_connections[0]
last = start
steps = 1

loop.add(start)
loop.add(current)

while current != start:
    for connection in connections[current]:
        if connection != last:
            last = current
            current = connection
            loop.add(current)
            steps += 1
            break
print(f'The farthest point from the starting position is {steps//2} steps away')

enclosed = 0
last_hit = None
for row in range(size[0]):
    inside = False
    for column in range(size[1]):
        if (row, column) in loop:
            c = lines[row][column]
            if c == 'S':
                c = start_sign
            
            if c == '|':
                inside = not inside
            if c == 'F' or c == 'L':
                last_hit = c
            if c == '7' and last_hit == 'L':
                inside = not inside
            if c == 'J' and last_hit == 'F':
                inside = not inside
        elif inside:
            enclosed += 1

print(f'There are {enclosed} tiles enclosed within the loop')

The farthest point from the starting position is 6923 steps away
There are 529 tiles enclosed within the loop


In [2]:
import itertools

input = open('input11.txt', 'r').readlines()
map = []
galaxy_counter = 1
for line in input:
    map_line = []
    for c in line:
        if c == '.':
            map_line.append(0)
        elif c == '#':
            map_line.append(galaxy_counter)
            galaxy_counter += 1
    map.append(map_line)

locations = {}
for row_index, row in enumerate(map):
    for column_index, column in enumerate(row):
        if column != 0:
            locations[int(column)] = (row_index, column_index)

empty_rows = [index for index, row in enumerate(map) if sum(row) == 0]
no_of_columns = len(map[0])
empty_columns = [i for i in range(no_of_columns) if sum(row[i] for row in map) == 0]
no_of_columns += len(empty_columns)

# Add empty columns
def expand(locations, expanded_rows, expanded_columns, expansion_size):
    expanded_locations = {}
    for location in locations:
        coordinate = locations[location]
        row_expansions = sum(1 if row < coordinate[0] else 0 for row in expanded_rows)
        column_expansions = sum(1 if column < coordinate[1] else 0 for column in expanded_columns)

        expanded_locations[location] = (coordinate[0] + expansion_size * row_expansions, coordinate[1] + expansion_size * column_expansions)
    return expanded_locations

all_pairs = [(i,j) for i in range(1,len(locations)+1) for j in range(i+1, len(locations)+1)]
small_expansion = expand(locations, empty_rows, empty_columns, 1)
large_expansion = expand(locations, empty_rows, empty_columns, 999999)

def find_distance_between_pair(pair, locations):
    location_one = locations[pair[0]]
    location_two = locations[pair[1]]

    return abs(location_one[0] - location_two[0]) + abs(location_one[1] - location_two[1])

print(f'The sum of distances between all galaxy pairs is {sum(find_distance_between_pair(pair, small_expansion) for pair in all_pairs)}')
print(f'The sum of distances between all galaxy pairs in the greatly expanded universe is {sum(find_distance_between_pair(pair, large_expansion) for pair in all_pairs)}')

The sum of distances between all galaxy pairs is 10165598
The sum of distances between all galaxy pairs in the greatly expanded universe is 678728808158


In [38]:
input = open('input12.txt', 'r').readlines()

def find_combinations(string, runs, seen):
    if len(runs) == 0:
        if '#' in string:
            return 0
        else:
            return 1

    combinations = 0
    run_length = runs[0]
    for i in range(len(string) - run_length+1):
        if i > 0 and string[i-1] == '#':
            break
        test_string = string[i:i+run_length+1]
        if '.' not in test_string[:run_length] and (len(test_string) == run_length or test_string[-1] != '#'):
            next_string = string[i+run_length+1:]
            next_runs = runs[1:]
            if (next_string,) + tuple(next_runs) not in seen:
                this_combinations = find_combinations(next_string, next_runs, seen)
                seen[(next_string,) + tuple(next_runs)] = this_combinations
            else:
                this_combinations = seen[(next_string,) + tuple(next_runs)]
            combinations += this_combinations

    return combinations

number_of_arrangements = []
number_of_unfolded_arrangements = []
for line_index, line in enumerate(input):
    string, numbers = line.split()
    runs = [int(x) for x in numbers.split(',')]
    seen = {}
    number_of_combinations = find_combinations(string, runs, seen)
    number_of_arrangements.append(number_of_combinations)

    long_string = ((string + '?')*5)[:-1]
    long_runs = runs*5

    number_of_combinations = find_combinations(long_string, long_runs, seen)
    number_of_unfolded_arrangements.append(number_of_combinations)
    print(f'\rOn line {line_index+1} of {len(input)}', end='')

print(f'\nThe total number of arrangements over all strings is {sum(number_of_arrangements)}')
print(f'The total number of unfolded arrangements is {sum(number_of_unfolded_arrangements)}')

On line 1000 of 1000
The total number of arrangements over all strings is 7732
The total number of unfolded arrangements is 4500070301581


In [53]:
areas = open('input13.txt', 'r').read().split('\n\n')
vertical_lines = 0
horizontal_lines = 0

smudge_vertical_lines = 0
smudge_horizontal_lines = 0

def find_horizontal_reflection(lines, old_line=None):
    for i in range(1, len(lines)):
        reflected = True
        for j in range(i):
            if i+j >= len(lines):
                break
            if lines[i-j-1] != lines[i+j]:
                reflected = False
                break

        if reflected and (old_line is None or old_line != i):
            return i

    return None

def find_area_reflections(area, old_horizontal=None, old_vertical = None):
    area_lines = area.strip().split('\n')
    horizontal = find_horizontal_reflection(area_lines, old_horizontal)
    flipped = [''.join(line[i] for line in area_lines) for i in range(len(area_lines[0]))]
    vertical = find_horizontal_reflection(flipped, old_vertical)
    return horizontal, vertical

for area in areas:
    horizontal, vertical = find_area_reflections(area)
    if horizontal is not None:
        horizontal_lines += horizontal
    if vertical is not None:
        vertical_lines += vertical

    # Smudge correction
    found = False
    for i in range(len(area)):
        new_area = None
        if area[i] == '.':
            new_area = area[:i] + '#' + area[i+1:]
        elif area[i] == '#':
            new_area = area[:i] + '.' + area[i+1:]
        else:
            continue

        new_horizontal, new_vertical = find_area_reflections(new_area, horizontal, vertical)
        if new_horizontal is not None:
            smudge_horizontal_lines += new_horizontal
            found = True
            break
        if new_vertical is not None:
            smudge_vertical_lines += new_vertical
            found = True
            break
    if not found:
        print(area)
        print()

print(f'The solution number is {100*horizontal_lines + vertical_lines}')
print(f'The solution number when correcting for smudges is {100*smudge_horizontal_lines + smudge_vertical_lines}')

The solution number is 37025
The solution number when correcting for smudges is 32854


In [8]:
input = '''O....#....
O.OO#....#
.....##...
OO.#O....O
.O.....O#.
O.#..O.#.#
..O..#O..O
.......O..
#....###..
#OO..#....'''.split('\n')
input = open('input14.txt', 'r').readlines()
platform_height = len(input)
platform_width = len(input[0].strip())
solid = set()
round = set()
for row, line in enumerate(input):
    for column, character in enumerate(line):
        if character == '#':
            solid.add((row, column))
        elif character == 'O':
            round.add((row, column))

def roll_north(round):
    rolled = set()
    for j in range(platform_width):
        last_solid = -1
        no_rocks = 0
        for i in range(platform_height):
            if (i, j) in round:
                no_rocks += 1
            if (i,j) in solid:
                for k in range(no_rocks):
                    rolled.add((last_solid+k+1, j))
                last_solid = i
                no_rocks = 0
        for k in range(no_rocks):
            rolled.add((last_solid+k+1, j))
    return rolled

def roll_west(round):
    rolled = set()
    for i in range(platform_height):
        last_solid = -1
        no_rocks = 0
        for j in range(platform_width):
            if (i, j) in round:
                no_rocks += 1
            if (i, j) in solid:
                for k in range(no_rocks):
                    rolled.add((i, last_solid + k + 1))
                last_solid = j
                no_rocks = 0
        for k in range(no_rocks):
            rolled.add((i, last_solid+k+1))
    return rolled

def roll_south(round):
    rolled = set()
    for j in range(platform_width):
        last_solid = platform_height
        no_rocks = 0
        for i in range(platform_height-1, -1, -1):
            if (i, j) in round:
                no_rocks += 1
            if (i, j) in solid:
                for k in range(no_rocks):
                    rolled.add((last_solid-k-1, j))
                no_rocks = 0
                last_solid = i
        for k in range(no_rocks):
            rolled.add((last_solid-k-1, j))
    return rolled

def roll_east(round):
    rolled = set()
    for i in range(platform_height):
        no_rocks = 0
        last_solid = platform_width
        for j in range(platform_width-1, -1, -1):
            if (i, j) in round:
                no_rocks += 1
            if (i, j) in solid:
                for k in range(no_rocks):
                    rolled.add((i, last_solid-k-1))
                no_rocks = 0
                last_solid = j
        for k in range(no_rocks):
            rolled.add((i, last_solid-k-1))
    return rolled

def calculate_load(round):
    total_load = 0
    for rock in round:
        total_load += platform_height - rock[0]
    return total_load

new_round = roll_north(roll_south(round))
total_load = calculate_load(new_round)
print(f'The total load on the platform is {total_load}')

cycle_start = None
cycle_length = None
seen = [round]

for i in range(1000000000):
    round = roll_east(roll_south(roll_west(roll_north(round))))
    if round in seen:
        cycle_start = seen.index(round)
        cycle_length = i + 1 - cycle_start
        break
    seen.append(round)

print(f'After 1000000000 cycles, the load is {calculate_load(seen[(1000000000 - cycle_start) % cycle_length + cycle_start])}')


The total load on the platform is 110274
After 1000000000 cycles, the load is 90982


In [46]:
import re
input = open('input15.txt', 'r').read().replace('\n','').split(',')

def hash_string(string):
    return sum(ord(c)*17**(i+1) for i, c in enumerate(string[::-1])) % 256

boxes = {i: {} for i in range(256)}
for label, focal_length in [re.split('[-=]', line) for line in input]:
    box_no = hash_string(label)
    if focal_length != '':
        boxes[box_no][label] = int(focal_length)
    elif label in boxes[box_no]:
        del boxes[box_no][label]
        
focusing_power = sum((box_no + 1) * (label_no + 1) * boxes[box_no][label] for box_no in boxes for label_no, label in enumerate(boxes[box_no]))
        
print(f'The sum of the hashes of each step is {sum(hash_string(step) for step in input)}')
print(f'The total focusing power is {focusing_power}')

The sum of the hashes of each step is 504036
The total focusing power is 295719
